In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
from pyha_analyzer import PyhaTrainer, PyhaTrainingArguments, extractors

# Loading in Data

In [3]:
# Sorry TQ, I don't have these files mounted yet...
# peru132_extr = extractors.Peru132Extractor()
# peru_132_ads = peru132_extr("/data/XC_wav")

In [4]:
# peru_132_ads = peru_132_ads.get_provenance()

In [5]:
birdset_extactor = extractors.Birdset()

hsn_ads = birdset_extactor("HSN")
# per_ads = birdset_extactor("PER")

hsn_ads

Map:   0%|          | 0/5460 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5460 [00:00<?, ? examples/s]

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/12000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 4368
    })
    valid: Dataset({
        features: ['audio', 'filepath', 'start_time', 'end_time', 'low_freq', 'high_freq', 'ebird_code', 'ebird_code_multilabel', 'ebird_code_secondary', 'call_type', 'sex', 'lat', 'long', 'length', 'microphone', 'license', 'source', 'local_time', 'detected_events', 'event_cluster', 'peaks', 'quality', 'recordist', 'genus', 'species_group', 'order', 'genus_multilabel', 'species_group_multilabel', 'order_multilabel', 'audio_in', 'labels'],
        num_rows: 1092
    }

In [19]:
import librosa

librosa.load("data_birdset/background_noise/fff12db0-9cbe-4155-ac4a-b0b88d84c1d7.wav")

(array([ 0.19588947,  0.27712345,  0.20782189, ..., -0.04795212,
        -0.12272076, -0.17080563], shape=(220500,), dtype=float32),
 22050)

# Online Preprocessing example

Suppose we just wanted spectrograms with no audio preprocessing

In [20]:
hsn_ads["test"][0]["audio"]

{'bytes': None,
 'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e7a5318118cabfab47a509edeb627860a60537535aeea20a19fced4c110d579e/HSN_001_20150708_061805_000_005.ogg'}

In [21]:
hsn_ads["train"][0]

{'audio': {'bytes': None,
  'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg'},
 'filepath': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg',
 'start_time': None,
 'end_time': None,
 'low_freq': None,
 'high_freq': None,
 'ebird_code': 16,
 'ebird_code_multilabel': [16],
 'ebird_code_secondary': [],
 'call_type': 'call, phrases',
 'sex': None,
 'lat': 31.34,
 'long': -109.273,
 'length': 58,
 'microphone': 'focal',
 'license': '//creativecommons.org/licenses/by-nc-sa/3.0/',
 'source': 'xenocanto',
 'local_time': '08:54',
 'detected_events': [[31.44, 33.792], [35.952, 36.672]],
 'event_cluster': [-1, -1],
 'peaks': [4.41303361102967,
  5.9407844283959825,
  7.327410170258567,
  10.157911684613236,
  13.236382081634444,
  19.7893855875755,
  25.044450899102944,
  26.92607690579703,
  28.70

In [22]:
hsn_ads["train"][0]

{'audio': {'bytes': None,
  'path': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg'},
 'filepath': '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg',
 'start_time': None,
 'end_time': None,
 'low_freq': None,
 'high_freq': None,
 'ebird_code': 16,
 'ebird_code_multilabel': [16],
 'ebird_code_secondary': [],
 'call_type': 'call, phrases',
 'sex': None,
 'lat': 31.34,
 'long': -109.273,
 'length': 58,
 'microphone': 'focal',
 'license': '//creativecommons.org/licenses/by-nc-sa/3.0/',
 'source': 'xenocanto',
 'local_time': '08:54',
 'detected_events': [[31.44, 33.792], [35.952, 36.672]],
 'event_cluster': [-1, -1],
 'peaks': [4.41303361102967,
  5.9407844283959825,
  7.327410170258567,
  10.157911684613236,
  13.236382081634444,
  19.7893855875755,
  25.044450899102944,
  26.92607690579703,
  28.70

In [23]:
from pyha_analyzer.preprocessors import MelSpectrogramPreprocessors

# TODO: allow for normalization system

# preprocessor acts as a function for processing
# class allows us to configure parameters and whatnot
preprocessor = MelSpectrogramPreprocessors(duration=5, class_list=hsn_ads["train"].features["labels"].feature.names)

# Set split spefific transforms here
# If preprocessor had data augmentations, probably want to disable for training
hsn_ads["train"].set_transform(preprocessor)
hsn_ads["valid"].set_transform(preprocessor)
hsn_ads["test"].set_transform(preprocessor)
hsn_ads["train"][[0, 1]]["audio"][0].mean()

np.float32(0.00093351223)

In [24]:
hsn_ads["train"][[0, 1]]

{'audio': [array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],
        shape=(1, 256, 431), dtype=float32),
  array([[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]]],
        shape=(1, 256, 431), dtype=float32)],
 'filepath': ['/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/e8d87da059276237fe3a09f216dfe50ac2186afa53bf9c3b9e14c93979cbeba4/XC123854.ogg',
  '/home/s.perry.543/.cache/huggingface/datasets/downloads/extracted/053832b39de8ba5377824eb2d8364a1e31d9355ede4974f3741a1daa9822e8ab/XC700958.ogg'],
 'start_time': [None, None],
 'end_time': [None, None],
 'low_f

# Model Training

As a demo, implementing a model here because we haven't fleshed out the AudioDataset api yet

In the future this exists in `/pyha_analyzer/models/`

In [25]:
# from pyha_analyzer.models.demo_CNN import ResnetConfig, ResnetModel

# # Going to make notes on anything that should be handled not here
# # This is one of these things, this should be handled by potentially pyha_trainer
# resnet50d_config = ResnetConfig(
#     num_classes=len(hsn_ads["train"].features["ebird_code"].names), input_channels=1
# )
# model = ResnetModel(resnet50d_config)
from pyha_analyzer.models import EfficentNet
model = EfficentNet(num_classes=len(hsn_ads["train"].features["ebird_code"].names))

In [26]:
args = PyhaTrainingArguments(
    working_dir="working_dir"
)
args.num_train_epochs = 1
args.eval_steps = 20


trainer = PyhaTrainer(
    model=model,
    dataset=hsn_ads,
    training_args=args
)
trainer.train()

wandb: Currently logged in as: shperry (acoustic-species-identification) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


KeyboardInterrupt: 

In [ ]:
trainer.evaluate(eval_dataset=hsn_ads["test"], metric_key_prefix="Soundscape")

KeyboardInterrupt: 

#